# Valorización de OIS

Entre sí, los bancos operan los instrumentos estándar. En el caso de los OIS sobre SOFR, éstos son instrumentos con periodicidad anual, amortización bullet y a los plazos predefinidos.

¿Qué se hace cuándo es necesario cotizar un swap con características distintas? En esos casos, se hace *pricing* de este contrato especial, utilizando la curva cupón cero que se ha obtenido de los swaps estándar.

## Configuración

### Librerías

In [1]:
from finrisk import QC_Financial_3 as Qcf
from scipy.optimize import root_scalar
import modules.auxiliary as aux
from functools import partial
from enum import Enum
import pandas as pd

### Variables Globales

In [2]:
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

## Carga Curva Cero Cupón

Se importa la data de la curva cupón cero que fue construida en el notebook 5.

In [3]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [4]:
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


In [5]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Algunos métodos del objeto`zcc`.

In [6]:
plazo = 900
print(f"Tasa a {plazo} días es igual a {zcc.get_rate_at(plazo):.4%}")
print(f"Factor de descuento a {plazo} días es igual a {zcc.get_discount_factor_at(plazo):.6%}")

Tasa a 900 días es igual a 0.0652%
Factor de descuento a 900 días es igual a 99.839384%


## Pricing

In [7]:
get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)

### Operación con Plazo Distinto

In [8]:
op = get_ois_sofr(
    rp=Qcf.RecPay.RECEIVE,
    notional=10000000,
    start_date=Qcf.QCDate(14, 10, 2020),
    tenor=Qcf.Tenor('2Y6M'),
    fixed_rate_value=.01,
    spread=0.0,
    gearing=1.0
)
op

(<finrisk.QC_Financial_3.Leg at 0x7fcde58ab168>,
 <finrisk.QC_Financial_3.Leg at 0x7fcde58ab238>)

In [9]:
aux.show_leg(op[0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-04-14,2021-04-14,"10,000,000.00",0.00,"50,555.56",True,"50,555.56",USD,1.0000%,LinAct360
1,2021-04-14,2022-04-14,2022-04-14,"10,000,000.00",0.00,"101,388.89",True,"101,388.89",USD,1.0000%,LinAct360
2,2022-04-14,2023-04-14,2023-04-14,"10,000,000.00","10,000,000.00","101,388.89",True,"10,101,388.89",USD,1.0000%,LinAct360


In [10]:
aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-04-14,2021-04-14,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
1,2021-04-14,2022-04-14,2022-04-14,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
2,2022-04-14,2023-04-14,2023-04-14,"-10,000,000.00","-10,000,000.00",True,"-10,000,000.00",CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360


#### Valor Presente Pata Fija

In [11]:
vp = Qcf.PresentValue()

In [12]:
fecha_val = Qcf.QCDate(14, 10, 2020)

In [13]:
vp_fija = vp.pv(fecha_val, op[0], zcc)
print(f'El valor presente de la pata fija es: USD {vp_fija:,.2f}')

El valor presente de la pata fija es: USD 10,236,626.11


#### Valor Presente Pata Flotante

In [14]:
fwd = Qcf.ForwardRates()

In [15]:
fwd.set_rates_icp_clp_leg(fecha_val, 1.0, op[1], zcc)

In [16]:
aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-04-14,2021-04-14,"-10,000,000.00",0.00,True,"-3,739.71",CLP,1.000000,1.000374,0.0700%,"-3,538.89",0.0000%,1.000000,LinAct360
1,2021-04-14,2022-04-14,2022-04-14,"-10,000,000.00",0.00,True,"-5,874.97",CLP,1.000374,1.000962,0.0600%,"-6,083.33",0.0000%,1.000000,LinAct360
2,2022-04-14,2023-04-14,2023-04-14,"-10,000,000.00","-10,000,000.00",True,"-10,006,827.97",CLP,1.000962,1.001645,0.0700%,"-7,097.22",0.0000%,1.000000,LinAct360


In [17]:
vp_flot = vp.pv(fecha_val, op[1], zcc)
print(f'El valor presente de la pata flotante es: USD {vp_flot:,.2f}')

El valor presente de la pata flotante es: USD -10,000,000.00


In [18]:
print(f'Por lo tanto, el valor total de la operación es:\nValor total: USD {vp_fija + vp_flot:,.2f}')

Por lo tanto, el valor total de la operación es:
Valor total: USD 236,626.11


#### Ejercicio

Haga el *pricing* de la operación: determine qué tasa fija hace que el valor total de la operación sea 0.

In [19]:
get_ois_sofr_solo_tasa = partial(
    aux.get_ois_using_template,
    template=aux.type_ois_template,
    rp=Qcf.RecPay.RECEIVE,
    type_ois=aux.TypeOis.SOFR,
    notional=10000000,
    start_date=Qcf.QCDate(14, 10, 2020),
    tenor=Qcf.Tenor('2Y6M'),
    spread=0.0,
    gearing=1.0
)

In [20]:
get_ois_sofr_solo_tasa(fixed_rate_value=0.0)

(<finrisk.QC_Financial_3.Leg at 0x7fcde58d6370>,
 <finrisk.QC_Financial_3.Leg at 0x7fcde58d63d8>)

In [21]:
def error(fixed_rate_value: float) -> float:
    this_op = get_ois_sofr_solo_tasa(fixed_rate_value=fixed_rate_value)
    err = vp.pv(fecha_val, this_op[0], zcc) - 10000000
    return err

In [22]:
tasa = .01
print(f'Error: {error(tasa):,.2f}')

Error: 236,626.11


In [23]:
x = root_scalar(
        error,
        method='bisect',
        bracket=[0.0, .02],
        x0=.01,
        xtol=.00000000000000001
)

In [24]:
print(f'Tasa fija es: {x.root:.8%}')

Tasa fija es: 0.06490559%


### Operación con Amortizaciones

In [25]:
op2 = get_ois_sofr(
    rp=Qcf.RecPay.RECEIVE,
    notional=10000000,
    start_date=Qcf.QCDate(14, 10, 2020),
    tenor=Qcf.Tenor('4Y'),
    fixed_rate_value=.01,
    spread=0.0,
    gearing=1.0
)
op2

(<finrisk.QC_Financial_3.Leg at 0x7fcde5861168>,
 <finrisk.QC_Financial_3.Leg at 0x7fcde58611d0>)

In [26]:
aux.show_leg(op2[0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000.00",0.00,"101,388.89",True,"101,388.89",USD,1.0000%,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"10,000,000.00",0.00,"101,388.89",True,"101,388.89",USD,1.0000%,LinAct360
2,2022-10-14,2023-10-16,2023-10-16,"10,000,000.00",0.00,"101,944.44",True,"101,944.44",USD,1.0000%,LinAct360
3,2023-10-16,2024-10-15,2024-10-15,"10,000,000.00","10,000,000.00","101,388.89",True,"10,101,388.89",USD,1.0000%,LinAct360


In [27]:
aux.show_leg(op2[1], 'IcpClpCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
2,2022-10-14,2023-10-16,2023-10-16,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
3,2023-10-16,2024-10-15,2024-10-15,"-10,000,000.00","-10,000,000.00",True,"-10,000,000.00",CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360


#### Agrega Amortizaciones

Vamos a agregar una amortización por la mitad del nocional en el segundo cupón de cada pata. Veamos primero la pata fija.

In [28]:
cshflw = op2[0].get_cashflow_at(1)
cshflw.set_amortization(5000000)

In [29]:
aux.show_leg(op2[0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000.00",0.00,"101,388.89",True,"101,388.89",USD,1.0000%,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"10,000,000.00","5,000,000.00","101,388.89",True,"5,101,388.89",USD,1.0000%,LinAct360
2,2022-10-14,2023-10-16,2023-10-16,"10,000,000.00",0.00,"101,944.44",True,"101,944.44",USD,1.0000%,LinAct360
3,2023-10-16,2024-10-15,2024-10-15,"10,000,000.00","10,000,000.00","101,388.89",True,"10,101,388.89",USD,1.0000%,LinAct360


Vemos que la amortización queda bien ingresada, sin embargo, los flujos siguientes no la consideran. Vamos a arreglar eso:

In [30]:
cshflw = op2[0].get_cashflow_at(2)
cshflw.set_nominal(5000000)

cshflw = op2[0].get_cashflow_at(3)
cshflw.set_nominal(5000000)
cshflw.set_amortization(5000000)

In [31]:
aux.show_leg(op2[0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000.00",0.00,"101,388.89",True,"101,388.89",USD,1.0000%,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"10,000,000.00","5,000,000.00","101,388.89",True,"5,101,388.89",USD,1.0000%,LinAct360
2,2022-10-14,2023-10-16,2023-10-16,"5,000,000.00",0.00,"50,972.22",True,"50,972.22",USD,1.0000%,LinAct360
3,2023-10-16,2024-10-15,2024-10-15,"5,000,000.00","5,000,000.00","50,694.44",True,"5,050,694.44",USD,1.0000%,LinAct360


Hagamos ahora la pata flotante. Dado que esta es la pata que pagamos, hay que usar signo negativo para los montos de amortización y capital vigente.

In [32]:
cshflw = op2[1].get_cashflow_at(1)
cshflw.set_amortization(-5000000)

cshflw = op2[1].get_cashflow_at(2)
cshflw.set_nominal(-5000000)

cshflw = op2[1].get_cashflow_at(3)
cshflw.set_nominal(-5000000)
cshflw.set_amortization(-5000000)

In [33]:
aux.show_leg(op2[1], 'IcpClpCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"-10,000,000.00","-5,000,000.00",True,"-5,000,000.00",CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
2,2022-10-14,2023-10-16,2023-10-16,"-5,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
3,2023-10-16,2024-10-15,2024-10-15,"-5,000,000.00","-5,000,000.00",True,"-5,000,000.00",CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360


#### Valor Presente por Pata

Calculemos ahora los valores presente de cada pata.

In [34]:
vp_fija_2 = vp.pv(fecha_val, op2[0], zcc)
print(f'El valor presente de la pata fija es: USD {vp_fija_2:,.2f}')

El valor presente de la pata fija es: USD 10,273,112.26


In [35]:
fwd.set_rates_icp_clp_leg(fecha_val, 1.0, op2[1], zcc)
vp_flot_2 = vp.pv(fecha_val, op2[1], zcc)
print(f'El valor presente de la pata flotante es: USD {vp_flot_2:,.2f}')

El valor presente de la pata flotante es: USD -10,000,000.00


In [36]:
print(f'Por lo tanto, el valor total de la operación es:\nValor total: USD {vp_fija_2 + vp_flot_2:,.2f}')

Por lo tanto, el valor total de la operación es:
Valor total: USD 273,112.26


#### Ejercicio

Pricee esta operación.

Segunda forma de obtener una función de 1 sola variable. Se define una función que **retorna** una función de 1 sola variable (que representa el valor de la tasa fija de la operación).

In [49]:
def get_sofr_ois_builder_only_rate(
    rp,
    notional,
    start_date,
    tenor,
    spread,
    gearing
):
    def f(fixed_rate_value):
        return get_ois_sofr(
            rp,
            notional,
            start_date,
            tenor,
            fixed_rate_value,
            spread,
            gearing
        )
    return f

Utilizamos la función anterior y obtenemos una función que construye un OIS SOFR a partir del valor de la tasa fija.

In [50]:
get_ois_sofr_solo_tasa2 = get_sofr_ois_builder_only_rate(
    rp=Qcf.RecPay.RECEIVE,
    notional=10000000,
    start_date=Qcf.QCDate(14, 10, 2020),
    tenor=Qcf.Tenor('4Y'),
    spread=0.0,
    gearing=1.0
)

Con la ayuda de la función anterior se obtiene una función `error2` que será la que utilizaremos para el *pricing*.

In [44]:
def error2(fixed_rate_value: float) -> float:
    this_op = get_ois_sofr_solo_tasa2(fixed_rate_value=fixed_rate_value)
    
    cshflw = this_op[0].get_cashflow_at(1)
    cshflw.set_amortization(5000000)
    
    cshflw = this_op[0].get_cashflow_at(2)
    cshflw.set_nominal(5000000)

    cshflw = this_op[0].get_cashflow_at(3)
    cshflw.set_nominal(5000000)
    cshflw.set_amortization(5000000)
    
    err = vp.pv(fecha_val, this_op[0], zcc) - 10000000
    return err

Se prueba la función `error2`.

In [45]:
tasa = .01
print(f'Error: {error2(tasa):,.2f}')

Error: 273,112.26


Se hace el *pricing*.

In [47]:
x2 = root_scalar(
        error2,
        method='bisect',
        bracket=[0.0, .02],
        x0=.01,
        xtol=.00000000000000001
)

In [48]:
print(f'Tasa fija es: {x2.root:.8%}')

Tasa fija es: 0.10127625%
